In [1]:
#pip3 install notebook ipywidgets plotly
#jupyter nbextension enable --py widgetsnbextension
#jupyter nbextension enable --py plotlywidget

In [2]:
import igraph as ig
import pandas as pd
import webbrowser
import json
import urllib.request
import chart_studio.plotly as py
import plotly.graph_objs as go
from ipywidgets import widgets
from scipy.special import softmax

This visualization is inspired from plotly officiel tutorials: https://plot.ly/python/v3/3d-network-graph/

In [3]:
# Load some data
data = []
req = urllib.request.Request("https://raw.githubusercontent.com/plotly/datasets/master/miserables.json")
opener = urllib.request.build_opener()
f = opener.open(req)
data = json.loads(f.read())

In [4]:
N = len(data['nodes'])
df_node = pd.DataFrame(data['nodes'])
df_node.drop(columns=['group'],axis=1,inplace=True)
df_node['url'] = 'https://fr.wikipedia.org/wiki/In_Deep'
df_edge = pd.DataFrame(data['links'])
df_edge.drop(columns=['value'],axis=1,inplace=True)

In [5]:
df_node.head()

,name,url
0,Myriel,https://fr.wikipedia.org/wiki/In_Deep
1,Napoleon,https://fr.wikipedia.org/wiki/In_Deep
2,Mlle.Baptistine,https://fr.wikipedia.org/wiki/In_Deep
3,Mme.Magloire,https://fr.wikipedia.org/wiki/In_Deep
4,CountessdeLo,https://fr.wikipedia.org/wiki/In_Deep


In [6]:
df_edge.head()

,source,target
0,1,0
1,2,0
2,3,0
3,3,2
4,4,0


In [7]:
# Extract list
labels = df_node['name'].tolist()
color_node = [0.0 for i in range(len(labels))]
texts_to_show = [None for i in range(len(labels))]
urls = df_node['url'].tolist()
Edges = df_edge[['source','target']].values.tolist()

G = ig.Graph(Edges, directed=False)
layt = G.layout('kk', dim=3)

In [8]:
# Coordinates for the nodes and edges
Xn = [layt[k][0] for k in range(N)]
Yn = [layt[k][1] for k in range(N)]
Zn = [layt[k][2] for k in range(N)]
Xe = []
Ye = []
Ze = []
for e in Edges:
    Xe += [layt[e[0]][0],layt[e[1]][0], None]
    Ye += [layt[e[0]][1],layt[e[1]][1], None]
    Ze += [layt[e[0]][2],layt[e[1]][2], None]

In [9]:
# Plot the edges
trace1 = go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
# Plot the nodes
trace2 = go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers+text',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=color_node,
                             colorscale='Reds',
                             line=dict(color='rgb(50,50,50)', width=0.5),
                               colorbar=dict(
                                    title=""
                                ),
                             ),
               text=texts_to_show,
               textposition="top center",
               hovertext=labels,
               hoverinfo='text',   
               customdata=urls
       )

# Set up the axis
axis = dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

# Custom layout
layout = go.Layout(
         title="",
         width=900,
         height=900,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
  )

# Create Figure
g = go.FigureWidget(data=[trace1, trace2],layout=layout)

In [10]:
# Title of the visualization
title = widgets.HTML(
    value="<h3> Wikipedia Recommender System </h3>",
)

# Some help text
annotations = widgets.HTML(
    value="<h4> By clicking on a node, you will be directed on the corresponding web page. </h4>",
)

In [11]:
# Compute color for the nodes
# Return a dict where the key is node index and the value is a float for the colorscale
def compute_color(preds):
    all_scores = list(preds.values())
    color_values = softmax(all_scores)
    color_output = {}
    diff = max(color_values)
    for k,v in zip(preds.keys(),color_values):
        color_output[k] = v/diff
    return color_output

In [12]:
# Create HTML formatted text for the answer of the query
def create_text(keys_ls,df_node):
    if len(keys_ls) == 0:
        "No pages found"
    else:
        base_text = "The most prominent pages are :<ul>"
        for key in keys_ls:
            filtered = df_node[df_node.index == key][['name','url']].values[0]
            url = filtered[1]
            name = filtered[0]
            url_text = "<li><a href=" + url + """ target="_blank"> """ + name + "</a></li>"
            base_text += url_text
        base_text += "</ul>"
    return base_text

In [13]:
# Text Box for the query
textbox_query = widgets.Text(
    value='',
    placeholder='Type something',
    description='Query:',
    disabled=False
)

# Select Method
selector = widgets.Select(
    options=['Node2Vec', 'Spectral Clustering', 'Node2Vec (NLP)'],
    value='Node2Vec',
    description='Method:',
    disabled=False
)

query_answer = widgets.HTML(value='')

In [14]:
# On a new query, compute the predictions and color the nodes accordingly
def response(change):
    current_selector = selector.value
    query = textbox_query.value
#     preds = make_prediction(query,current_selector,topk=5)
    preds = {10:1,11:0.8,12:0.6,13:0.4,14:0.3}
    dict_colors = compute_color(preds)
   
    html_text = create_text(list(dict_colors.keys()),df_node)
    query_answer.value = html_text
    
    for k,v in dict_colors.items():
        color_node[k] = v
        texts_to_show[k] = labels[k]
        
    with g.batch_update():
        g.data[1].marker.color = color_node
        g.data[1].text = texts_to_show

textbox_query.observe(response, names="value")
selector.observe(response, names="value")

# Open url when clicking on node
def update_point(trace, points, selector):
    if len(points.point_inds) != 0:
        url = g.data[1].customdata[points.point_inds[0]]
        webbrowser.open_new_tab(url)

g.data[1].on_click(update_point)

In [15]:
container = widgets.HBox(children=[textbox_query, selector])
widgets.VBox([title, annotations, container, query_answer, g])